### Alpha Blending
> Tutorial: https://github.com/enomotokenji/mcgan-cvprw2017-pytorch/blob/master/data.py

In [79]:
from glob import glob
import cv2
import random
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
from torchvision import transforms
import pdb
import random
from PIL import Image

import scipy.io
import skimage.io

from osgeo import gdal
gdal.UseExceptions()

### Single Image Test

In [80]:
def readTif(fileName):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName + "文件无法打开")
    return dataset

def writeTiff(im_data, im_geotrans, im_proj, path):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    elif len(im_data.shape) == 2:
        im_data = np.array([im_data])
        im_bands, im_height, im_width = im_data.shape
    #创建文件
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(path, int(im_width), int(im_height), int(im_bands), datatype)
    if(dataset!= None):
        dataset.SetGeoTransform(im_geotrans) #写入仿射变换参数
        dataset.SetProjection(im_proj) #写入投影
    for i in range(im_bands):
        dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
    del dataset

In [81]:
def truncated_linear_stretch(
    image, truncated_percent=2, stretch_range=[0, 255], is_drop_non_positive=False
):
    """_summary_

    Args:
        image (np.array): HWC or HW
        truncated_percent (int, optional): _description_. Defaults to 2.
        stretch_range (list, optional): _description_. Defaults to [0, 255].
    """
    max_tansformed_img = (
        np.where(image <= 0, 65536, image) if is_drop_non_positive else image
    )
    min_tansformed_img = (
        np.where(image <= 0, -65536, image) if is_drop_non_positive else image
    )

    truncated_lower = np.percentile(
        max_tansformed_img, truncated_percent, axis=(0, 1), keepdims=True
    )
    truncated_upper = np.percentile(
        min_tansformed_img, 100 - truncated_percent, axis=(0, 1), keepdims=True
    )

    stretched_img = (image - truncated_lower) / (truncated_upper - truncated_lower) * (
        stretch_range[1] - stretch_range[0]
    ) + stretch_range[0]
    stretched_img[stretched_img < stretch_range[0]] = stretch_range[0]
    stretched_img[stretched_img > stretch_range[1]] = stretch_range[1]
    if stretch_range[1] <= 255:
        stretched_img = np.uint8(stretched_img)
    elif stretch_range[1] <= 65535:
        stretched_img = np.uint16(stretched_img)
    return stretched_img

#### Clouds checking + Blending (RGB)

If need cloud, please [email](yujunguo2018@163.com)

In [ ]:
img_tf = transforms.Compose([transforms.ToPILImage(), transforms.Resize(size=(256, 256))])
save_dir = 'file to savefolder for simulated cloudy image'
img_path = 'file to underlying clean image'
cld_path = 'file to simulated cloud'

img = cv2.imread(img_path)
# img = (img_tf(img))
cld = cv2.imread(cld_path, -1)
cld = np.asarray(img_tf(cld))
name = img_path.split('\\')[-1]
save_path = os.path.join(save_dir, name)

# Alpha Blending
alpha = cld[:, :, 3] / 255.
alpha = np.broadcast_to(alpha[:, :, None], alpha.shape + (3,))
cld_img = (1. - alpha) * img + alpha * cld[:, :, :3]
cld_img = np.clip(cld_img, 0., 255.)

plt.subplot(1, 3, 1)
plt.imshow(img)

plt.subplot(1, 3, 2)
plt.imshow(cld[:, :, 3])

plt.subplot(1, 3, 3)
plt.imshow(cld_img / 255)

cv2.imwrite(save_path, cld_img)

### Multiple Simulated Cloudy Images Generation

#### Underlying Image: 13bands

In [ ]:

save_dir_cly = ''
save_dir_cld = ''
if not os.path.exists(save_dir_cly):
    os.makedirs(save_dir_cly)
if not os.path.exists(save_dir_cld):
    os.makedirs(save_dir_cld)

img_tf = transforms.Compose([transforms.ToPILImage(), transforms.Resize(size=(256, 256))])
img_paths = glob('/*.tif', recursive=True)
cld_paths0 = glob('/*.png', recursive=True) 
cld_paths1 = glob('/*.png', recursive=True)
cld_paths2 = glob('/*.png', recursive=True)
l = len(img_paths)
c = len(cld_paths2)
cld_paths0 = cld_paths0[:c]
cld_paths1 = cld_paths1[:c]
cld_paths2 = cld_paths2[:c]

for i, path in enumerate(img_paths):
    if i < c:
            cld0 = cv2.imread(cld_paths0[i], -1)
            cld1 = cv2.imread(cld_paths1[i], -1)
            cld2 = cv2.imread(cld_paths2[i], -1)
    else:
        n = random.randint(0, c-1)
        cld0 = cv2.imread(cld_paths0[n], -1)
        cld1 = cv2.imread(cld_paths1[n], -1)
        cld2 = cv2.imread(cld_paths2[n], -1)

    cld0 = np.asarray(img_tf(cld0))
    cld013 = np.concatenate((cld0, cld0, cld0, cld0[:, :, 1:3]), axis=-1)
    cld1 = np.asarray(img_tf(cld1))
    cld113 = np.concatenate((cld1, cld1, cld1, cld1[:, :, 1:3]), axis=-1)
    cld2 = np.asarray(img_tf(cld2))
    cld213 = np.concatenate((cld2, cld2, cld2, cld2[:, :, 1:3]), axis=-1)
    noncld = np.zeros(cld0.shape)
    noncld13 = np.concatenate((noncld, noncld, noncld, noncld[:, :, 1:3]), axis=-1)

    # no cloud
    name = os.path.basename(path)  
    dataset = readTif(path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    proj = dataset.GetProjection()
    geotrans = dataset.GetGeoTransform()
    img = dataset.ReadAsArray(0, 0, width, height) # 获取数据
    img_t = np.transpose(img, (1, 2, 0))
    img_t= truncated_linear_stretch(img_t, 2, [0, 255])
    img_t = np.transpose(img_t, (2, 0, 1))
    save_path_cly = os.path.join(save_dir_cly, '{0}'.format(name))
    save_path_cld = os.path.join(save_dir_cld, '{0}'.format(name))
    writeTiff(img_t, geotrans, proj, save_path_cly)
    writeTiff(noncld13, geotrans, proj, save_path_cld)

    # cloud
    name = os.path.basename(path)  
    dataset = readTif(path)
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    proj = dataset.GetProjection()
    geotrans = dataset.GetGeoTransform()
    img = dataset.ReadAsArray(0, 0, width, height) # 获取数据
    img_t = np.transpose(img, (1, 2, 0))
    img_t= truncated_linear_stretch(img_t, 2, [0, 255])
    # Alpha Blending(四通道)
    alpha0 = cld0[:, :, 3] / 255.
    alpha0 = np.broadcast_to(alpha0[:, :, None], alpha0.shape + (13,))
    cld_img = (1. - alpha0) * img_t + alpha0 * cld013[:, :, :13]
    cld_img = np.clip(cld_img, 0., 255.)
    # Save
    cld_img_t = np.transpose(cld_img, (2, 0, 1))
    save_path_cly = os.path.join(save_dir_cly, '{0}'.format(name))
    save_path_cld = os.path.join(save_dir_cld, '{0}'.format(name))
    writeTiff(cld_img_t, geotrans, proj, save_path_cly)
    writeTiff(cld013, geotrans, proj, save_path_cld)


Visualize

In [ ]:
dataset = readTif('')
width = dataset.RasterXSize
height = dataset.RasterYSize
proj = dataset.GetProjection()
geotrans = dataset.GetGeoTransform()
img = dataset.ReadAsArray(0, 0, width, height) # 获取数据
# img_t = np.transpose(img, (1, 2, 0))
# img_RGB= truncated_linear_stretch(img_[t:,:,[4,5,6]], 2, [0, 255])
img_RGB= truncated_linear_stretch(img[:,:,3], 2, [0, 255])
plt.imshow(img_RGB)

##### Cloud Mask

In [98]:
from glob import glob
import cv2
import random
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
from torchvision import transforms

#### Underlying Image: RGB

In [ ]:
dir = ''
mode = 1

cld_path = ''.format(name)
img_path = ''.format(name)
save_dir = ''
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

if mode == 0:
    save_path = os.path.join(save_dir, 'mask0_{0}.png'.format(name))
elif mode == 1:
    save_path = os.path.join(save_dir, 'mask1_{0}.png'.format(name))

dataset = readTif(cld_path)
width = dataset.RasterXSize
height = dataset.RasterYSize
proj = dataset.GetProjection()
geotrans = dataset.GetGeoTransform()
img = dataset.ReadAsArray(0, 0, width, height) # 获取数据
img_t= truncated_linear_stretch(img, 2, [0, 255])
cloud = img_t[:,:,3:4]
means = np.mean(cloud)
mask = np.zeros((256, 256, 1))
# cloud 0
if mode == 0:
    mask = np.where(cloud>=means/1.2, mask, 255)
    mask = np.where(cloud<=means/2.6, mask, 0)
# cloud 1
elif mode == 1:
    mask = np.where(cloud<=means/1.2, mask, 255)
    mask = np.where(cloud>=means/2.6, mask, 0)

cv2.imwrite(save_path, mask)

 

#### Underlying Image: 13 Bands

In [ ]:
dir = ''

cld_paths = glob('', recursive=True)
img_paths = glob('', recursive=True)
# save_dir = '{0}/cloud_mask_0'.format(dir)
save_dir = '{0}/cloud_mask_1'.format(dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

l = len(cld_paths)
for i in range(l):
    name = os.path.basename(cld_paths[i]).split('.')[0]
    temp = name.split('_')[-1]
    save_path = os.path.join(save_dir, '{0}.png'.format(name))

    dataset = readTif(cld_paths[i])
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    proj = dataset.GetProjection()
    geotrans = dataset.GetGeoTransform()
    img = dataset.ReadAsArray(0, 0, width, height) # 获取数据
    img_t= truncated_linear_stretch(img, 2, [0, 255])
    cloud = img_t[:,:,3:4]
    means = np.mean(cloud)
    mask = np.zeros((256, 256, 1))
    # # cloud 0
    # if temp == 'temp0':
    #     mask = np.ones((256, 256, 1))*255.
    # mask = np.where(cloud>=means/1.2, mask, 255)
    # mask = np.where(cloud<=means/2.6, mask, 0)
    # cloud 1
    if temp == 'temp0':
        mask = np.zeros((256, 256, 1))
    mask = np.where(cloud<=means/1.2, mask, 255)
    mask = np.where(cloud>=means/2.6, mask, 0)

    cv2.imwrite(save_path, mask)

    # img_t = np.transpose(img_t, (2, 0, 1))
    # writeTiff(img_t, geotrans, proj, savepath_clean)

##### Tiff to Mat

In [ ]:
import scipy.io as sio
import skimage.io

savedir = ''
if not os.path.exists(savedir):
    os.makedirs(savedir)

paths = glob('')
names = ['_'.join(os.path.basename(path).split('_')[0:2]) for path in paths]
b = 13
t = 3
for name in names:
    clean = np.zeros((256, 256, b, t))
    cloudy = np.zeros((256, 256, b, t))
    mask = np.zeros((256, 256, b, t))
    for i in range(t):
        path_cloudy = '/{0}_{1}.tif'.format(name, (i+1))
        path_clean = '/{0}_temp0.tif'.format(name)
        path_mask = '/{0}_temp{2}.png'.format(mode, name, (i+1)) 
        savepath = '{0}/{1}.mat'.format(savedir, name)

        dataset_cloudy = readTif(path_cloudy)
        width = dataset_cloudy.RasterXSize
        height = dataset_cloudy.RasterYSize
        proj = dataset_cloudy.GetProjection()
        geotrans = dataset_cloudy.GetGeoTransform()
        img = dataset_cloudy.ReadAsArray(0, 0, width, height) # 获取数据
        img_t = np.transpose(img, (1, 2, 0))
        img_t= truncated_linear_stretch(img_t, 2, [0, 255])
        cloudy[:,:,:,i] = img_t

        dataset_clean = readTif(path_clean)
        width = dataset_clean.RasterXSize
        height = dataset_clean.RasterYSize
        proj = dataset_clean.GetProjection()
        geotrans = dataset_clean.GetGeoTransform()
        img = dataset_clean.ReadAsArray(0, 0, width, height) # 获取数据
        img_t = np.transpose(img, (1, 2, 0))
        img_t= truncated_linear_stretch(img_t, 2, [0, 255])
        clean[:,:,:,i] = img_t

        for j in range(b):
            img_t = np.array(Image.open(path_mask))
            # img_t = cv2.imread(path_mask)
            mask[:,:,j,i] = img_t
    
    temps = {'cleans':clean, 'cloudys':cloudy, 'masks1':mask}
    sio.savemat(savepath, temps)   